In [ ]:
import requests
import psycopg2
from psycopg2 import sql

# Partidos

In [ ]:
# Fazendo a requisição

url = 'https://dadosabertos.camara.leg.br/api/v2/partidos'

params = {'idLegislatura': '56', 'ordenarPor': 'sigla', 'itens': 100}


try:
    response = requests.get(url, params=params)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Erro ao fazer a requisição: {e}")
    exit()

data = response.json()

# Conectando ao banco de dados
try:
    with psycopg2.connect(
        dbname="pi2",
        user="postgres",
        password="2605",
        host="localhost",
        port="5432"
    ) as conn:
        with conn.cursor() as cur:
            # Criando a tabela
            table_create_query = """
            CREATE TABLE IF NOT EXISTS partidos (
                sigla VARCHAR(50) PRIMARY KEY,
                nome VARCHAR(255)
            )
            """
            cur.execute(table_create_query)

            # Inserindo os dados na tabela
            for partido in data['dados']:
                insert_query = """
                INSERT INTO partidos (sigla, nome) 
                VALUES (%s, %s)
                """
                cur.execute(insert_query, (partido['sigla'], partido['nome']))
            conn.commit()
except psycopg2.Error as e:
    print(f"Erro ao conectar ao banco de dados: {e}")
    exit()

# Deputados

In [ ]:
# Fazendo a requisição

url = 'https://dadosabertos.camara.leg.br/api/v2/deputados'

params = {'idLegislatura': '56', 'ordenarPor': 'id'}

# params = {'dataInicio':'2020-01-01','dataFim':'2020-12-31'}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
except requests.exceptions.RequestException as e:
    print(f"Erro ao fazer a requisição: {e}")
    exit()

data = response.json()

# Conectando ao banco de dados
try:
    with psycopg2.connect(
        dbname="pi2",
        user="postgres",
        password="2605",
        host="localhost",
        port="5432"
    ) as conn:
        with conn.cursor() as cur:
            # Criando a tabela
            table_create_query = """
            CREATE TABLE IF NOT EXISTS deputados (
                id SERIAL PRIMARY KEY,
                nome VARCHAR(255),
                partido VARCHAR(50) REFERENCES partidos(sigla)
            )
            """
            cur.execute(table_create_query)

            # Inserindo os dados na tabela
            for deputado in data['dados']:
                insert_query = sql.SQL("""
                    INSERT INTO deputados (id, nome, partido) 
                    VALUES (%s, %s, %s)
                    ON CONFLICT (id) DO NOTHING
                """)
                cur.execute(
                    insert_query, (deputado['id'], deputado['nome'], deputado['siglaPartido']))

            conn.commit()
except psycopg2.Error as e:
    print(f"Erro ao conectar ao banco de dados: {e}")

# Gastos

In [ ]:
# Conectando ao banco de dados
with psycopg2.connect(
    dbname="pi2",
    user="postgres",
    password="2605",
    host="localhost",
    port="5432"
) as conn:
    with conn.cursor() as cur:
        # Criando a tabela gastos se ela não existir
        cur.execute("""
            CREATE TABLE IF NOT EXISTS gastos (
                id SERIAL PRIMARY KEY,
                id_deputado INTEGER REFERENCES deputados(id),
                data DATE,
                valor FLOAT,
                tipoDespesa VARCHAR(255),
                codDocumento INTEGER,
                partido VARCHAR(50) REFERENCES partidos(sigla)
            )
        """)
        conn.commit()  # Confirmar a transação após a criação da tabela

        # Selecionando todos os IDs de deputados e seus partidos
        cur.execute("SELECT id, partido FROM deputados")
        deputados = cur.fetchall()
        # Para cada deputado
        Count = 0
        for deputado in deputados:
            id_deputado = deputado[0]
            partido = deputado[1]  # Buscando o partido do deputado
            pagina = 1
            Count += 1
            print(Count)
            while True:
                # Fazendo a requisição para a API
                url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id_deputado}/despesas'
                params = {'ano': '2020', 'pagina': pagina,
                          'itens': 100, 'ordenarPor': 'dataDocumento'}
                response = requests.get(url, params=params)
                data = response.json()
                # Se não houver mais dados, sair do loop
                if not data['dados']:
                    break

                # Para cada gasto do deputado
                for gasto in data['dados']:
                    cur.execute("""
                        INSERT INTO gastos (id_deputado, data, valor, tipoDespesa, codDocumento, partido) 
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, (id_deputado, gasto['dataDocumento'], gasto['valorLiquido'], gasto['tipoDespesa'], gasto['codDocumento'], partido))
                    conn.commit()  # Confirmar a transação após cada inserção

                # Ir para a próxima página
                pagina += 1
        # Fechando o cursor e a conexão
        cur.close()
        conn.close()

# Separador

In [ ]:
# Conectando ao banco de dados
with psycopg2.connect(
    dbname="pi2",
    user="postgres",
    password="2605",
    host="localhost",
    port="5432"
)as conn:
    with conn.cursor() as cur:
        # Criando a tabela gastos se ela não existir

        cur.execute("""
            CREATE TABLE IF NOT EXISTS gastos_estranhos (
                id SERIAL PRIMARY KEY,
                id_deputado INTEGER REFERENCES deputados(id),
                data DATE,
                valor FLOAT,
                tipoDespesa VARCHAR(255),
                codDocumento INTEGER,
                partido VARCHAR(50) REFERENCES partidos(sigla)
            )
        """)
        conn.commit()

        # Encontrando as linhas com valores estranhos
        cur.execute("""
            SELECT * FROM gastos WHERE TO_CHAR(data, 'YYYY') != '2020' OR codDocumento = 0 OR valor <= 0
        """)
        rows = cur.fetchall()

        # Inserindo as linhas estranhas na nova tabela e removendo da tabela original
        for row in rows:
            cur.execute("""
                INSERT INTO gastos_estranhos (id_deputado, data, valor, tipoDespesa, codDocumento, partido) 
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (row[1], row[2], row[3], row[4], row[5], row[6]))
            cur.execute("""
                DELETE FROM gastos WHERE id = %s
            """, (row[0],))
            conn.commit()

        # Fechando o cursor e a conexão
    cur.close()
    conn.close()